In [1]:
from functools import cached_property
import re

TODAY = 'day15'
TEST_FILE_INPUT = f"./test_input_{TODAY}.txt"
FILE_INPUT = f"./input_{TODAY}.txt"


In [17]:
class PartA:
    def __init__(self, file_path):
        self.file_path = file_path
        
    @cached_property
    def string_list(self):
        with open(self.file_path, 'r') as f:
            line = f.readline()
            line = line.rstrip('\n')
            return line.split(',')
        
    def get_hash(self, in_str):
        current_value = 0
        for char in in_str:
            current_value += ord(char)
            current_value *= 17
            current_value = current_value % 256
        return current_value
        

    def solve(self):
        return sum([self.get_hash(in_str) for in_str in self.string_list])

In [18]:
a_test = PartA(TEST_FILE_INPUT)


In [19]:
assert a_test.solve() == 1320 # NUMER HERE

In [20]:
a = PartA(FILE_INPUT)
a.solve()

513172

In [142]:
class Lens:
    def __init__(self, label, focal_length):
        self.label = label
        self.focal_length = focal_length
        
    def __str__(self):
        return f'[{self.label} {self.focal_length}]'
        
    def same_label(self, other_lens):
        return self.label == other_lens.label

class Box:
    def __init__(self, box_number, slots):
        self.box_number = box_number
        self.slots = slots
        
    def __str__(self):
        out_str = '['
        for lens in self.slots:
            out_str += str(lens)
        out_str += ']'
        return out_str
        
    def remove(self, lens_label):
        found_match = None
        for idx, lens in enumerate(self.slots):
            if lens.label == lens_label:
                found_match = idx
        if found_match is not None:
            self.slots = self.slots[:found_match] + self.slots[found_match+1:]
            
    def insert(self, new_lens):
        replacement_idx = None
        for idx, lens in enumerate(self.slots):
            if new_lens.same_label(lens):
                replacement_idx = idx
        if replacement_idx is not None:
            self.slots = self.slots[:replacement_idx] + [new_lens] + self.slots[replacement_idx+1:]
        else:
            self.slots += [new_lens]


class PartB(PartA):
    def __init__(self, file_path):
        self.file_path = file_path
        self.boxes = {idx: Box(box_number=idx, slots = []) for idx in range(256)}        
        
    def perform_operations(self):
        for instruct_string in self.string_list:
            if instruct_string.endswith('-'):
                label = instruct_string[:-1]
                operation = 'remove'
                
            else:
                label = instruct_string[:-2]
                operation = 'insert'
                focal_length = int(instruct_string[-1])
                new_lens = Lens(label, focal_length)
                
            box_position = self.get_hash(label)
            
            if operation == 'remove':
                self.boxes[box_position].remove(label)
                
            else:
                self.boxes[box_position].insert(new_lens)
                
    @cached_property
    def focusing_power(self):
        power = 0
        for box_num, box in self.boxes.items():
            box_weight = box_num + 1
            slots_weight = 0
            for idx, lens in enumerate(box.slots):
                slots_weight += (idx + 1) * lens.focal_length
                
            box_contribution = box_weight * slots_weight
            # print(f"Box {box.box_number}: {box_contribution}")
            power += box_contribution
            
        return power
        
    def solve(self):
        self.perform_operations()
        return self.focusing_power

                

In [143]:
b_test = PartB(TEST_FILE_INPUT)

In [145]:
assert b_test.solve() == 145 # NUMBER HERE

In [146]:
b = PartB(FILE_INPUT)
b.solve()

237806